In [1]:
import pandas as pd
trump_raw = pd.read_csv("C:/Users/ariver10/Downloads/archive/hashtag_donaldtrump.csv",
                 lineterminator='\n')

biden_raw = pd.read_csv("C:/Users/ariver10/Downloads/archive/hashtag_joebiden.csv",
                 lineterminator='\n')


In [ ]:
#ONLY RUN IF you are making the full non-location DataFrame This block to clean the full Trump dataset
#Removes columns of data with very high Na values or of limited utility
trump=trump_raw.drop(['user_description','user_location','user_name', 'user_screen_name','long','lat','source','city','country','continent',
'state','state_code', 'collected_at'], axis=1)

#Removes columns of data with very high Na values or of limited utility

print(f"This data has (rows x col){trump.shape}")
print(f"total NaN values in this dataFrame: {trump.isna().sum().sum()}")
trump.dtypes

In [ ]:
#ONLY RUN IF you are making the full non-location DataFrame.  This block to clean the full Biden dataset
#Removes columns of data with very high Na values or of limited utility
biden=biden_raw.drop(['user_description','user_location','user_name', 'user_screen_name','long','lat','source','city','country','continent',
'state','state_code', 'collected_at'], axis=1)

print(f"This data has (rows x col){biden.shape}")
print(f"total NaN values in this dataFrame: {biden.isna().sum().sum()}")
biden.dtypes

In [9]:
#START HERE for location set makes new dataset with locations (only for tweets with location data)
tweets_loc = trump_raw.append(biden_raw) #adds trump and biden hashtag data into a single dataframe shape = (1747805, 21)
tweets_loc = tweets_loc.drop_duplicates(subset = 'tweet_id') #drops overlapping tweets (hashtag for both Trump and Biden) shape = (1522909, 21)
US = tweets_loc['country'] == 'United States of America' #selects only USA tweets
tweets_loc = tweets_loc[US] #shape = (301341, 21)
tweets_loc=tweets_loc.drop(['user_description','user_location','user_name', 'user_screen_name','source','city','country','continent',
 'collected_at'], axis=1) #drops extra columns we wont use shape = (301341, 10)
tweets_loc = tweets_loc.reset_index()

In [ ]:
#DO NOT RUN unless you are making the countylist through geolocator API
#READ BEFORE YOU RUN This block queries the online geopy database.  Only run it if you are adding in a small set of samples = 1 second per query
# makes countylist 
latitude = list(tweets_loc['lat'])
longitude = list(tweets_loc['long'])
countylist = []
for j in range(len(latitude)):
    lat = latitude[j] #set lat and long from dataframe
    long = longitude[j] 
    loc = geolocator.reverse("{}, {}".format(lat, long), timeout=10) #reverse location with geopy
    address = loc.raw.get('address') #pull address (which is a dictionary) from location data
    addin = address.get('county') #pull county from address
    countylist.append(addin)

    tweets_loc['count'] = countylist #adds countlist into tweets_loc dataset

# Adaptation
1) makes new column based on time until the election

In [10]:
#converts created_at to a numerical value of time until the election
from datetime import datetime
import dateutil.parser as dparser
created_at = tweets_loc['created_at']
countdown = []
for i in range(len(created_at)):
    time = dparser.parse(created_at[i])
    election = datetime(2020, 11, 3)
    newtime = (election - time).days
    countdown.append(newtime)
tweets_loc['countdown'] = countdown

In [11]:
tweets_loc.head()

,index,created_at,tweet_id,tweet,likes,retweet_count,user_id,user_join_date,user_followers_count,lat,long,state,state_code,countdown
0,0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,3.606665e+08,2011-08-23 15:33:45,1860.0,25.774270,-80.193660,Florida,FL,18
1,2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,8.436472e+06,2007-08-26 05:56:11,1185.0,45.520247,-122.674195,Oregon,OR,18
2,4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,4.741380e+07,2009-06-15 19:05:35,5393.0,38.894992,-77.036558,District of Columbia,DC,18
3,5,2020-10-15 00:00:17,1.316529e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,1.138416e+09,2013-02-01 01:37:38,2363.0,33.782519,-117.228648,California,CA,18
4,7,2020-10-15 00:00:18,1.316529e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,9.007611e+17,2017-08-24 16:45:49,766.0,40.225357,-82.688140,Ohio,OH,18


In [12]:
joined = tweets_loc['user_join_date']
account = tweets_loc['created_at']
agelist = []
for i in range(len(joined)):
    time = dparser.parse(joined[i])
    created = dparser.parse(account[i])
    newtime = (created-time).days
    agelist.append(newtime)
tweets_loc['age'] = agelist
tweets_loc = tweets_loc.drop(['created_at', 'index', 'user_join_date'], axis = 1)

In [13]:
tweets_loc.head()

,tweet_id,tweet,likes,retweet_count,user_id,user_followers_count,lat,long,state,state_code,countdown,age
0,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,3.606665e+08,1860.0,25.774270,-80.193660,Florida,FL,18,3340
1,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,8.436472e+06,1185.0,45.520247,-122.674195,Oregon,OR,18,4798
2,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,4.741380e+07,5393.0,38.894992,-77.036558,District of Columbia,DC,18,4139
3,1.316529e+18,@CLady62 Her 15 minutes were over long time ag...,2.0,0.0,1.138416e+09,2363.0,33.782519,-117.228648,California,CA,18,2812
4,1.316529e+18,@DeeviousDenise @realDonaldTrump @nypost There...,0.0,0.0,9.007611e+17,766.0,40.225357,-82.688140,Ohio,OH,18,1147


In [8]:
#DO NOT RUN unless you are making the county list
import reverse_geocoder as rg
import pprint

In [ ]:
#DO NOT RUN unless you are making the county list
#READ BEFORE YOU RUN THIS: Takes about 0.75 sec/query on a laptop or 0.2 sec/query on Colab.  Only run if you need to!  
#The resultant csv is countylist.csv, please use that instead
latitude = list(tweets_loc['lat'])
longitude = list(tweets_loc['long'])
countylist = []
for j in range(len(latitude)):
    coordinates = (latitude[j], longitude[j])
    loc = rg.search(coordinates) #reverse location with geopy
    if loc[0].get('admin2') =='':
        addin = loc[0].get('name')
    else:
        addin = loc[0].get('admin2') #pull county from address
    countylist.append(addin)
tweets_loc['county'] = countylist

In [ ]:
#DO NOT RUN YET, DATASET INCOMPLETE
#alternative to above using already run dataset
counties = pd.read_csv("path.to/countylist.csv")
tweets_loc['county'] = counties['countylist']